---
Logistic Regression on [MNIST](http://yann.lecun.com/exdb/mnist/) digits
---

train-images-idx3-ubyte.gz:  training set images (9912422 bytes)
train-labels-idx1-ubyte.gz:  training set labels (28881 bytes)
t10k-images-idx3-ubyte.gz:   test set images (1648877 bytes)
t10k-labels-idx1-ubyte.gz:   test set labels (4542 bytes)


In [2]:
import numpy as np
import matplotlib.pyplot as plt

# for confusion matrix
from sklearn import metrics
import seaborn as sns

# for loading MNIST
from struct import unpack

%matplotlib inline

In [12]:
import os

# Load downloaded dataset
data_root = '../../books1000'
def loadMNIST(imagefile, labelfile):
    #img_filename = os.path.join(data_root, imagefile)
    #lbl_filename = os.path.join(data_root, labelfile)
    images = open(imagefile, 'rb')
    labels = open(labelfile, 'rb')
    
    images.read(4)
    number_of_images = images.read(4)
    number_of_images = unpack('>I', number_of_images)[0]
    rows = images.read(4)
    rows = unpack('>I', rows)[0]
    cols = images.read(4)
    cols = unpack('>I', cols)[0]
    
    labels.read(4)
    N = labels.read(4)
    N = unpack('>I', N)[0]
    
    x = np.zeros((N, rows*cols), dtype=np.uint8)
    y = np.zeros(N, dtype=np.uint8)
    
    for i in range(N):
        for j in range(rows*cols):
            tmp_pixel = images.read(1)
            tmp_pixel = unpack('>B', tmp_pixel)[0]
            x[i][j] = tmp_pixel
        tmp_label = labels.read(1)
        y[i] = unpack('>B', tmp_label)[0]
        
    images.close()
    labels.close()
    return (x, y)

In [14]:
train_img_filename = os.path.join(data_root, 'train-images-idx3-ubyte')
train_lbl_filename = os.path.join(data_root, 'train-labels-idx1-ubyte')
train_dataset, train_labels = loadMNIST(train_img_filename,
                                        train_lbl_filename)

test_img_filename = os.path.join(data_root, 't10k-images-idx3-ubyte')
test_lbl_filename = os.path.join(data_root, 't10k-labels-idx1-ubyte')
test_dataset, test_labels = loadMNIST(test_img_filename,
                                      test_lbl_filename)

In [19]:
print(train_dataset.shape)
print(train_labels.shape)

print(test_dataset.shape)
print(test_labels.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


## Train a Logistic Regression model using [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html). 

We will split the data into 5 sets and perform five-fold cross validation. This will help generalize the model.

In [26]:
# Import
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# Instantiate
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', random_state=42, verbose=1,
                           max_iter=1000, n_jobs=-1)

scores = cross_val_score(model, train_dataset, train_labels, cv=5)
print(scores)
print(scores.mean())

[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/prakriti/anaconda/envs/ml_100/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.2min finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/prakriti/anaconda/envs/ml_100/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.3min finished
[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/prakriti/anaconda/envs/ml_100/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the nu

[0.91378592 0.90526579 0.90975    0.90339251 0.91905635]
0.9102501156960354


/Users/prakriti/anaconda/envs/ml_100/lib/python3.5/site-packages/sklearn/linear_model/logistic.py:757: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  3.0min finished


In [25]:
print(scores)
print(model)


[0.91378592 0.90526579 0.90975    0.90339251 0.91905635]
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=1000, multi_class='multinomial',
          n_jobs=-1, penalty='l2', random_state=42, solver='lbfgs',
          tol=0.0001, verbose=1, warm_start=False)


---
Model selection
---

Using cross validation we have multiple model. The following cell lets us select the best model from all the trained models using scikit-learn KFold and cross_validation_score. Later we will use LeNet to train a model for digit classification

### Testing different parameters to understand how accuracies change.
Understanding how decision regions change when using different regularization values.
Remember that we use paramter C as our regularization parameter. Parameter C = 1/λ.
Lambda (λ) controls the trade-off between allowing the model to increase it's complexity as much as it wants with trying to keep it simple. For example, if λ is very low or 0, the model will have enough power to increase it's complexity (overfit) by assigning big values to the weights for each parameter. If, in the other hand, we increase the value of λ, the model will tend to underfit, as the model will become too simple.
Parameter C will work the other way around. For small values of C, we increase the regularization strength which will create simple models which underfit the data. For big values of C, we low the power of regularization which imples the model is allowed to increase it's complexity, and therefore, overfit the data.¶

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.learning_curve import validation_curve

C_param_range = [0.001,0.01,0.1,1,10,100]

digit_acc_table = pd.DataFrame(columns = ['C_parameter','Accuracy'])
digit_acc_table['C_parameter'] = C_param_range

plt.figure(figsize=(10, 10))

j = 0
for i in C_param_range:
    
    # Apply logistic regression model to training data
    lr = LogisticRegression(penalty = 'l2', C = i,random_state = 0)
    lr.fit(train_dataset, train_labels)
    
    # Predict using model
    y_pred = lr.predict(test_dataset)
    
    # Saving accuracy score in table
    digit_acc_table.iloc[j,1] = accuracy_score(test_labels,y_pred)
    j += 1
    
    # Printing decision regions
    plt.subplot(3,2,j)
    plt.subplots_adjust(hspace = 0.4)
    #plot_decision_regions(X = X_combined_digit_standard
    #                  , y = Y_combined_digit
    #                  , classifier = lr
    #                  , test_idx = range(105,150))
    plt.xlabel('Digit length')
    plt.ylabel('Digit width')
    plt.title('C = %s'%i)